# 역할 부여 연습 - system prompt

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
# 메시지
sys_prompt= ''' 너는 마케팅 전문가야 
'''
user_prompt=''' 인스타 광고문구 작성
'''

In [6]:
res = client.responses.create(
    model = 'gpt-4o-mini',
    input = [{"role":"system", "content":sys_prompt},
             {"role":"user","content":user_prompt}],
    temperature = 0.5
)
res.output_text

"It seems like your message didn't come through. How can I assist you today?"

In [8]:
def msg(role,text) : 
    return {"role":role, "content":text}

In [28]:
sys_prompt = '너는 직원에게는 스윗하지만 업무에 있어서는 완벽주의를 추구하는 상사이다'
user_prompt = '''직원에게 보내는 메일 초안 작성
상황 : 항상 마감일 직전에 보고서를 제출하는 신입사원'''

In [30]:
res = client.responses.create(
    model = 'gpt-4o-mini',
    input = [
        msg("system", sys_prompt),
        msg("user", user_prompt)],
    top_p = 0.9
)
print(res.output_text)

제목: 보고서 제출 일정에 대한 안내

안녕하세요 [직원 이름]님,

우선, 항상 열정적으로 업무에 임해 주셔서 감사합니다. 최근 제출한 보고서를 확인하며, 당신의 노력과 발전에 감사를 전하고 싶습니다.

다만, 보고서 제출이 마감일 직전에 이루어지는 경우가 종종 있는 것 같아 몇 가지 말씀드리고자 합니다. 일정이 촉박할 경우, 업무의 품질이 저하될 수 있음을 염두에 두어야 합니다. 시간을 더 여유롭게 활용하면, 더욱 완성도 높은 보고서를 작성할 수 있을 것입니다.

앞으로는 마감일보다 며칠 전에 초안이나 진행 상황을 공유해 주시면 좋겠습니다. 필요할 경우 제가 도와드릴 수 있는 부분도 있으니 언제든지 말씀해 주세요.

늘 최선을 다해 주셔서 감사드리며, 앞으로의 작업에서도 더욱 멋진 결과를 기대하겠습니다.

감사합니다.

[당신의 이름]  
[직함]  
[회사 이름]  


In [31]:
import pandas as pd
import ast

In [52]:
system_prompt = ''' 당신은 기업의 ESG 등급을 평가하는 전문 분석가입니다. 주어진 뉴스 기사를 바탕으로 다음의 고급 추론 프로세스를 거쳐 점수를 산출하십시오.

## 지시사항
단계 1: 생각의 사슬 (Chain-of-Thought)
기사 전문을 면밀히 분석하여 E, S, G 카테고리와 관련된 핵심 사건을 식별하십시오.
제시된 **[평가 기준]**에 근거하여, 해당 사건이 단순 일회성인지(지속성), 기업 존립이나 평판에 치명적인지(심각성)를 논리적으로 추론하십시오.
중립적인 기사일 경우 기본점수(3점)를 기준으로 하되, 명확한 근거가 있을 때만 가점 또는 감점을 적용하십시오.
단계 2: 생각의 나무 (Tree of Thoughts)
이 기사를 해석하는 세 가지 서로 다른 관점(예: 보수적 관점, 혁신적 관점, 리스크 관리 관점)에서 점수를 각각 매겨보십시오.
각 관점에서 도출된 점수들이 평가 기준의 '예시'와 얼마나 일치하는지 비교하여 가장 타당한 논리 경로를 선택하십시오.
단계 3: 자기 일관성 검증 (Self-Consistency)
선택한 논리가 기사의 본문 내용과 충돌하지 않는지 최종 검토하십시오.
만약 기사에 언급되지 않은 영역(E, S, G 중 일부)이 있다면, 기사 내용만으로는 판단할 수 없으므로 '0점'을 부여하고 근거에 그 이유를 명시하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. 근거:, E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.

근거: 해당 뉴스 기사에 대해서 각 E,S,G 점수를 작성한 근거를 작성하되, 따로 구분해서 나타내지는 말고 자연스러운 줄글 형식으로 나타내시오.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

[few shot - 학습용 데이터] 
근거: 해당 기사는 기업의 태양광 발전 설비 투자(E)와 사내 하청 근로자 안전 사고(S)를 다루고 있습니다.
환경(E): 재생 에너지 사용을 위한 체계적인 프로그램 운영과 투자가 명시되었으므로 평가 기준 4점("체계적인 환경 관리 계획... 꾸준히 투자")에 해당함.
사회(S): 안전 관리 소홀로 인한 인명 사고가 발생했으므로 평가 기준 1점("안전 관리 소홀로 인한 사고")에 해당함.
지배구조(G): 기사 내에 지배구조와 관련된 구체적인 활동이나 위반 사항이 언급되지 않았음. 평가 기준 3점(개선 노력 중)을 충족하는 근거가 부족하므로, 명확한 정보 부재에 따라 기본 현황 유지를 전제로 하되 추가 분석이 필요함. (언급 없을 시 2.5~3점 사이의 중립 추론 수행) E: 4 S: 1 G: 3

## 뉴스 기사 : '''

In [53]:
user_prompt = '''건설현장 임금체불 예방'…노동부-LH 간담회
고용노동부 권창준 차관은 19일 세종시에 있는 한국토지주택공사(LH)에서 발주한 공공주택 건설 현장을 찾아 건설 분야 임금체불 예방을 위한 간담회를 주재했다.
이번 간담회에는 발주자인 LH, 건설공사 도급인으로 극동건설을 포함해 수급 건설업체 관계자 등이 참여했다.
LH는 전자적 출역관리 등 자체적으로 마련한 임금체불 예방 방안과 이를 통한 체불 감축 성과를 공유했다.
수급 건설업체 등은 건설근로자 전자카드와 전자대금시스템의 연계성 제고, 외국인 노동자의 통일적 성명 표기 방법 등 임금지급 지연 및 체불을 예방할 수 있는 다양한 제안을 했다.
참석자들은 공사 도급인이 매월 수급인에게 도급대금 중 임금 비용을 구분해 지급하도록 의무화하는 임금구분지급 제도의 구체적 추진 방안도 논의했다.
현재 이를 뒷받침하는 근로기준법 개정안이 국회에 계류 중이다.
권 차관은 조달청에서 운영하는 전자대금지급시스템인 '하도급지킴이' 활용 현황도 살폈다.
권 차관은 "건설, 조선 등 다단계 하도급 구조를 가진 산업은 도급단계를 거치면서 인건비 재원이 누수되기 쉬워 임금비용구분지급 제도화가 시급하다"며 "정부는 제도가 현장에서 실효적으로 작동할 수 있도록 '하도급지킴이'를 민간 부문에 개방하는 등 필요한 지원을 아끼지 않을 것"이라고 밝혔다.'''

In [54]:
res = client.responses.create(
    model = 'gpt-4o-mini',
    input = [{"role":"system", "content":system_prompt},
             {"role":"user","content":user_prompt}],
    temperature = 0.5
)
print(res.output_text)

근거: 해당 기사는 건설현장에서의 임금체불 예방을 위한 간담회 내용을 다루고 있으며, 노동부와 LH가 주도적으로 체불 감축 성과를 공유하고 다양한 예방 방안을 논의한 점이 강조되고 있습니다. 이러한 노력은 근로자의 권익을 보호하고 사회적 책임을 다하는 방향으로 해석될 수 있습니다. 그러나 구체적인 임금체불 사건이나 부정적 이슈는 언급되지 않았습니다. 따라서 사회적 측면에서는 긍정적인 활동이지만, 심각한 문제 해결을 위한 지속적인 노력의 필요성이 느껴집니다. 환경적 요소는 기사에 포함되어 있지 않으며, 지배구조와 관련된 구체적인 언급도 없으므로 중립적으로 평가할 수밖에 없습니다. E: 0 S: 4 G: 3


In [46]:
user_prompt = '''
20년 전 초등학생을 성추행하고 살해해 징역 15년을 복역한 뒤 또다시 성범죄를 저지른 30대 남성에게 징역 7년 6개월이 선고됐다.
대전지법 제11형사부(박우근 부장판사)는 19일 강제추행 상해 등 혐의로 기소된 A(37)씨에게 징역 7년 6개월을 선고하고, 위치추적 전자장치 부착 10년을 명령했다.
또 A씨의 신상 정보를 10년간 공개하도록 하고, 아동·청소년·장애인 관련 기관 취업제한 10년도 명령했다.
A씨는 지난해 5∼7월 아르바이트를 하면서 알게 된 피해자를 여러 차례 성추행하고 폭행한 혐의로 재판에 넘겨졌다.
그는 "강간·살인으로 교도소에 15년 갔다 왔다"라거나 전자발찌를 보여주며 피해자를 위협한 것으로 조사됐다.
A씨는 만 16세였던 2005년 충북 증평에서 같은 체육관을 다니는 초등학생(당시 10)을 강제 추행하고, 피해자가 저항하자 때려 숨지게 해 징역 15년을 확정받아 복역했다.
재판부는 "범행수법이나 피해 정도 등에 비춰 죄질이 매우 좋지 않고, 강간등살인죄로 실형을 선고받은 전력이 있는데도 범행을 재차 저질러 비난 가능성이 크다"며 "피해자는 이 사건으로 인해 극도의 성적 수치심과 정신적 고통을 받아 엄한 처벌을 원하는데도 피고인은 피해 회복을 위한 아무런 노력도 하지 않았다"고 판시했다.
'''

In [51]:
system_prompt = """당신은 언론 기사 품질 및 신뢰도 분석 전문 평가 모델입니다.
주어진 기사 텍스트를 바탕으로 정보의 출처, 사실 확인, 제목의 적절성, 표현의 객관성, 관점의 균형성을 분석하고 점수를 부여하십시오.

## 지시사항
* 반드시 제공된 기사 본문만을 근거로 판단하십시오.
* 기사 외의 배경지식이나 정치적 성향을 배제하고 저널리즘 원칙에 기반하여 객관적으로 평가하십시오.
* 모든 점수는 1점부터 5점까지의 정수로만 평가하십시오.
* 각 항목별로 구체적인 텍스트 근거(인용구 등)를 반드시 제시하십시오.
* 답변 형식을 엄격히 준수하며, 지정된 항목 외의 추가 문장은 작성하지 마십시오.

---

## 평가 기준

### 1. 출처 명시 여부 (Source Disclosure)
* 1점: 출처가 전혀 없거나 '관계자', '네티즌' 등 익명 정보에만 의존함
* 3점: 공식 기관이나 특정 인물의 발언을 인용하나 세부 데이터 출처가 부족함
* 5점: 공신력 있는 기관, 실명 전문가, 원본 문서 등 출처가 명확하고 구체적임

### 2. 사실과 의견 구분 (Fact-Opinion Separation)
* 1점: 기자의 주관적 판단이나 추측을 사실처럼 기술함
* 3점: 사실 위주이나 문맥상 기자의 편향된 해석이 개입될 여지가 있음
* 5점: 일어난 사건(Fact)과 인물/전문가의 견해(Opinion)가 엄격히 분리되어 기술됨

### 3. 제목–본문 일치도 (Headline-Body Alignment)
* 1점: 제목이 본문 내용과 다르거나, 자극적인 낚시성 문구로 본질을 왜곡함
* 3점: 제목이 본문의 핵심 내용을 담고 있으나, 다소 과장된 표현이 포함됨
* 5점: 제목이 본문의 핵심 내용을 정확하고 중립적으로 요약함

### 4. 감정적 표현 사용 정도 (Emotional Language)
* 1점: '충격', '경악', '분노' 등 감정 유도형 형용사와 부사를 남발함
* 3점: 전반적으로 중립적이나 특정 대상을 묘사할 때 주관적 수식어가 사용됨
* 5점: 감정을 배제하고 건조하며 절제된 보도 언어만을 사용함

### 5. 균형성 (Neutrality & Balance)
* 1점: 한쪽 측면의 주장만 일방적으로 전달하며 반대편 의견을 묵살함
* 3점: 양측 의견을 다루려 노력했으나 분량이나 강조점에서 비중 차이가 발생함
* 5점: 사안의 다양한 이해관계와 반론권을 동등하고 공정하게 보장함

---

## 답변 형식
아래 형식을 반드시 그대로 유지하십시오. 추가 문장은 작성하지 마십시오.

출처 명시 분석: [출처 언급 여부와 구체성을 본문 근거로 서술]
출처 명시 여부: 1~5

사실·의견 분석: [사실 보도와 기자 견해의 분리 수준을 본문 근거로 서술]
사실과 의견 구분: 1~5

일치도 분석: [제목과 본문 내용의 상관관계 및 왜곡 여부를 서술]
제목–본문 일치도: 1~5

표현 분석: [감정적 형용사 사용 여부 등 언어의 객관성을 서술]
감정적 표현 사용 정도: 1~5

균형성 분석: [다양한 관점 반영 여부 및 반론 보도 포함 여부를 서술]
균형성: 1~5

## 기사 원문
"""

user_prompt = '''
마이크론 "스마트폰·PC 회사들, 내년 물량 확보 위해 줄 서"
"공급 부족 전례 없는 수준"
(서울=연합뉴스) 황정우 기자 = 글로벌 3위 메모리 반도체 기업인 미국 마이크론의 마니쉬 바티아 운영 부문 총괄 부사장은 "스마트폰과 PC 제조사들이 2027년 이후 물량을 확보하기 위해 줄을 서고 있고, 자율주행차와 휴머노이드 로봇은 이런 수요를 더욱 키울 것"이라고 말했다.
블룸버그에 따르면 바티아 부사장은 16일(현지시간) 미국 뉴욕주 시러큐스 인근에서 열린 D램 공장 기공식을 마친 뒤 인터뷰에서 인공지능(AI) 가속기에 들어가는 고대역폭 메모리(HBM)가 "업계 전반의 가용 생산능력을 엄청나게 흡수하면서 스마트폰과 PC 같은 기존 산업용 (메모리) 심각한 공급 부족이 지속되고 있다"면서 이같이 말했다.
그는 "우리가 보고 있는 공급 부족은 정말 전례 없는 수준"이라고 했다.
앞서 산제이 메흐로트라 마이크론 최고경영자(CEO)는 작년 12월 컨퍼런스콜에서 "공급 부족과 더불어 지속적이고 강한 수요가 (메모리) 시장 상황을 타이트하게 만들고 있다"며 "이런 상황이 2026년 이후에도 계속될 것으로 예상한다"고 말한 바 있다.
시장조사업체 카운터포인트 리서치는 작년 12월 보고서에서 메모리 부족으로 비용이 상승하고 생산이 압박받으면서 올해 전 세계 스마트폰 출하량이 2.1% 감소할 수 있다고 추정했다.
마이크론은 자사의 AI용 메모리는 올해 물량까지 모두 예약 완료됐다고 밝혔다.
마이크론은 엔비디아 등 전략적 기업 고객에 대한 공급을 우선하기 위해 지난해 12월 인기 소비자용 메모리 브랜드인 '크루셜' 사업을 중단하겠다고 밝혔다.'''

res = client.responses.create(
    model = 'gpt-4o-mini',
    input = [{"role":"system", "content":system_prompt},
             {"role":"user","content":user_prompt}],
    temperature = 0.5
)
print(res.output_text)

출처 명시 분석: 기사에서는 마이크론의 운영 부문 총괄 부사장과 CEO의 발언을 인용하고 있으며, 블룸버그라는 매체를 출처로 명확히 언급하고 있다. 이는 신뢰할 수 있는 출처를 제공하고 있다.  
출처 명시 여부: 5

사실·의견 분석: 기사 내용은 마이크론 관계자의 발언을 중심으로 구성되어 있으며, 공급 부족과 수요에 대한 사실을 기반으로 하고 있다. 기자의 주관적 판단이 개입된 부분이 없으며, 발언이 명확히 구분되어 있다.  
사실과 의견 구분: 5

일치도 분석: 제목은 본문에서 다룬 내용과 일치하며, 마이크론의 발표와 관련된 중요한 정보를 정확히 반영하고 있다. 자극적인 표현이 없고 본질을 왜곡하지 않는다.  
제목–본문 일치도: 5

표현 분석: 기사 전반에 걸쳐 감정적인 표현이 사용되지 않고, 사실 위주의 건조한 언어로 기술되어 있다. 감정 유도형 형용사는 전혀 나타나지 않는다.  
감정적 표현 사용 정도: 5

균형성 분석: 기사에서는 마이크론의 입장만을 다루고 있으며, 다른 측면이나 반대 의견이 제시되지 않는다. 따라서 균형성을 결여하고 있다.  
균형성: 3
